# Лабабаторная работа №1: Регрессия


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import datetime
import random

Загрузка датасета и предобработка


In [2]:
data = pd.read_csv('../data/regression/diamonds.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
data = data.drop_duplicates()
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [4]:
data.isnull().sum() #hype

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [5]:
cat_features = ['cut', 'color','clarity']
prefixes = ['ct', 'clr', 'clrty']
data = pd.get_dummies(columns=cat_features, data=data, prefix=prefixes)
data

,carat,depth,table,price,x,y,z,ct_Fair,ct_Good,ct_Ideal,...,clr_I,clr_J,clrty_I1,clrty_IF,clrty_SI1,clrty_SI2,clrty_VS1,clrty_VS2,clrty_VVS1,clrty_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,0,1,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,0,1,...,0,0,0,0,1,0,0,0,0,0
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,0,1,0,...,0,0,0,0,1,0,0,0,0,0
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
y = data["price"]
X = data.drop(["price"], axis=1)
y

0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: price, Length: 53794, dtype: int64

In [7]:
X

,carat,depth,table,x,y,z,ct_Fair,ct_Good,ct_Ideal,ct_Premium,...,clr_I,clr_J,clrty_I1,clrty_IF,clrty_SI1,clrty_SI2,clrty_VS1,clrty_VS2,clrty_VVS1,clrty_VVS2
0,0.23,61.5,55.0,3.95,3.98,2.43,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,3.89,3.84,2.31,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,4.05,4.07,2.31,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,4.20,4.23,2.63,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,4.34,4.35,2.75,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,5.75,5.76,3.50,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
53936,0.72,63.1,55.0,5.69,5.75,3.61,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
53937,0.70,62.8,60.0,5.66,5.68,3.56,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53938,0.86,61.0,58.0,6.15,6.12,3.74,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((43035, 26), (43035,), (10759, 26), (10759,))

Линейная регрессия


In [9]:
lr = LinearRegression().fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [10]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 725.258603262385
MSE: 1215024.2243764668
RMSE: 1102.2813725979709
MAPE: 0.6160369344667002
R^2: 0.9202848296231219


In [11]:
len(lr.coef_)
lr.coef_

array([ 1.12718584e+04, -6.11323005e+01, -2.68549975e+01, -9.76109475e+02,
        3.62177081e+00, -1.02335211e+02,  1.67760254e+14,  1.67760254e+14,
        1.67760254e+14,  1.67760254e+14,  1.67760254e+14,  4.17551046e+13,
        4.17551046e+13,  4.17551046e+13,  4.17551046e+13,  4.17551046e+13,
        4.17551046e+13,  4.17551046e+13,  1.71540585e+14,  1.71540585e+14,
        1.71540585e+14,  1.71540585e+14,  1.71540585e+14,  1.71540585e+14,
        1.71540585e+14,  1.71540585e+14])

L1: Ridge


In [ ]:
ridge = Ridge().fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

L2: Lasso


In [349]:
lasso = Lasso().fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 736.0054755745198
MSE: 1288109.8305221999
RMSE: 1134.9492634132152
MAPE: 0.6256606174531414
R^2: 0.9189706046286295


array([ 1.11327560e+04, -6.63368867e+01, -2.78397301e+01, -9.62697162e+02,
       -2.05009519e+00, -2.24720384e+01, -7.20317365e+02, -1.47571229e+02,
        1.04580541e+02,  2.60335877e+01,  0.00000000e+00,  4.81029547e+02,
        2.67021175e+02,  2.06350693e+02,  0.00000000e+00, -4.96069958e+02,
       -9.69090759e+02, -1.85073757e+03, -4.28084268e+03,  9.73995532e+02,
       -6.81504431e+02, -1.64834009e+03,  2.10308917e+02, -9.20930094e+01,
        6.40569923e+02,  5.87555108e+02])

Удалим признаки, которые быстро занулились Лассо - улучшится качество модели


In [8]:
X_train = X_train.drop(X_train.columns[[10, 14]], axis=1)
X_test = X_test.drop(X_test.columns[[10, 14]], axis=1)
X_train

NameError: name 'X_train' is not defined

Прогоним Lasso & Ridge снова без гиперпарапара и подберем их


In [351]:
lasso = Lasso().fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 736.0291108816438
MSE: 1288107.1785377574
RMSE: 1134.9480950853028
MAPE: 0.6256623290113823
R^2: 0.9189707714534536


array([ 1.11300098e+04, -6.63265921e+01, -2.78438947e+01, -9.62018403e+02,
       -1.94765297e+00, -2.20136058e+01, -7.20371779e+02, -1.47611501e+02,
        1.04510482e+02,  2.60003414e+01,  4.80464890e+02,  2.66534698e+02,
        2.05923352e+02, -4.96376853e+02, -9.69313852e+02, -1.85090663e+03,
       -4.28062198e+03,  9.73956536e+02, -6.81490060e+02, -1.64825416e+03,
        2.10257310e+02, -9.20890692e+01,  6.40568229e+02,  5.87540614e+02])

In [352]:
parameters = {'alpha': np.arange(0, 10, 0.1)}

In [353]:
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.16394e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


{'alpha': 1.4000000000000001}

In [354]:
lasso_optimal = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)
lasso_optimal.best_params_

c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.235e+10, tolerance: 5.552e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_

{'alpha': 0.30000000000000004}

In [355]:
ridge = Ridge(alpha=1.4).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 737.5146512026342
MSE: 1288614.4127926216
RMSE: 1135.171534523581
MAPE: 0.6288148091234282
R^2: 0.918938863549322


array([ 1.12426190e+04, -6.44503072e+01, -2.65764230e+01, -9.92546552e+02,
       -3.36514630e+00, -3.69275249e+01, -7.50599081e+02, -1.58367751e+02,
        1.08799054e+02,  3.19138417e+01,  4.95571755e+02,  2.77542394e+02,
        2.15859578e+02, -5.02440145e+02, -9.82023157e+02, -1.87307701e+03,
       -3.81006163e+03,  1.54400733e+03, -1.46308667e+02, -1.11920709e+03,
        7.58202662e+02,  4.42129580e+02,  1.19426626e+03,  1.13697155e+03])

In [356]:
lasso = Lasso(alpha=0.3).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 736.7784658834198
MSE: 1288417.63424007
RMSE: 1135.0848577265358
MAPE: 0.6277896943042515
R^2: 0.9189512420334834


array([ 1.12355093e+04, -6.54677211e+01, -2.69735477e+01, -9.94282201e+02,
       -3.05711362e+00, -3.20253364e+01, -7.41074254e+02, -1.54990110e+02,
        1.07557230e+02,  3.01861342e+01,  4.91044640e+02,  2.74126528e+02,
        2.12881852e+02, -5.01364463e+02, -9.79592480e+02, -1.86877384e+03,
       -4.33838279e+03,  9.94140545e+02, -6.85861119e+02, -1.65766940e+03,
        2.14804667e+02, -9.73007671e+01,  6.48668282e+02,  5.92803901e+02])

Перейдем к полиномиальной регрессии


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [357]:
p = PolynomialFeatures(2)  
X_train=p.fit_transform(X_train)
X_test=p.fit_transform(X_test)
X_train.shape

(43152, 325)

In [358]:
pr = LinearRegression().fit(X_train, y_train)
y_pred = pr.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {pr.score(X_test, y_test)}')
pr.coef_

MAE: 401.61893272204316
MSE: 481159.2601833366
RMSE: 693.6564424723067
MAPE: 0.3989357776203794
R^2: 0.9697323605439871


array([ 4.44931549e-10,  8.97796245e+09,  3.94749737e+11,  1.38621010e+10,
        1.52128591e+10,  9.19196781e+09, -1.91398156e+10, -6.36217860e+07,
        1.91964568e+08, -1.85820956e+07, -2.00866899e+08, -9.71544360e+07,
       -1.18381083e+07, -1.73010551e+08,  1.46886196e+08,  2.65946549e+08,
        1.93025797e+08, -8.65468779e+07, -1.19495840e+08, -6.93004325e+07,
       -1.29890487e+07, -1.24886150e+08,  3.76475218e+08, -1.36240019e+08,
       -1.52349286e+08, -1.07013004e+03, -3.45852467e+01,  1.06870763e+02,
       -5.19056809e+03,  5.06370563e+03,  2.74418746e+03,  1.30690107e+03,
       -8.77966125e+02,  8.25801428e+02,  7.71972878e+02, -3.67429211e+01,
        5.78040068e+02,  2.76965795e+02, -8.45304730e+02, -1.25707147e+03,
       -2.49280046e+03, -8.97797401e+09, -8.97796238e+09, -8.97796455e+09,
       -8.97796692e+09, -8.97796317e+09, -8.97796386e+09, -8.97796137e+09,
       -8.97796203e+09, -2.32666016e+00, -6.51431274e+00,  1.88486267e+02,
       -7.48980179e+01, -

In [359]:
ridge = Ridge().fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 402.28875701077646
MSE: 471799.0345566654
RMSE: 686.8762876651555
MAPE: 0.3994790204877761
R^2: 0.9703211716881125


array([ 0.00000000e+00, -7.35573081e+02,  1.04513067e+02,  4.85384119e+02,
       -1.21276593e+03, -1.02087076e+03, -4.81554022e+02, -6.77339923e+01,
       -8.38727428e+02,  9.05671066e+02,  1.20497068e+02, -3.34590286e+02,
       -1.19756441e+03, -7.31876327e+01,  1.24506207e+02,  1.21302614e+03,
        8.40147092e+02, -1.35600042e+03, -3.88018684e+02,  8.99129795e+02,
        9.08047861e+02, -6.26332436e+02,  5.51581771e+02,  2.79907432e+02,
       -2.68315321e+02, -1.35011352e+03, -2.28390263e+01,  8.25554133e+01,
       -3.27647245e+03,  4.13377113e+03,  1.36207180e+03,  8.82754346e+02,
       -9.03826078e+02,  7.10073701e+02,  6.61865385e+02,  3.02282930e+01,
        5.92425902e+02,  2.35130781e+02, -8.39418917e+02, -1.27091560e+03,
       -2.42486392e+03, -7.61217514e+03,  1.86551602e+03,  3.58132819e+01,
       -2.25303800e+03,  1.36701861e+03,  6.52690142e+02,  2.87925419e+03,
        2.32934781e+03, -2.17019220e-01, -4.16890035e+00,  9.72992547e+01,
       -3.11259850e+01, -

In [360]:
lasso = Lasso().fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 414.2155825892055
MSE: 514009.7182253646
RMSE: 716.9447107171965
MAPE: 0.39363659510354493
R^2: 0.9676658808931495


c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.246e+10, tolerance: 6.870e+07
  model = cd_fast.enet_coordinate_descent(


array([ 0.00000000e+00,  3.06421196e+03,  1.53479613e+02,  1.34903886e+02,
        4.68531470e+02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -2.57788210e+02,  6.27834437e+01, -7.64552976e+00,
        7.81299873e+02,  1.53721416e+02,  0.00000000e+00, -5.32199849e+01,
       -2.19866528e+02,  8.22003071e+01,  0.00000000e+00,  0.00000000e+00,
        5.01560924e+02,  0.00000000e+00, -1.08244666e+03, -1.13873387e+03,
       -2.45199347e+03, -7.93573122e+03,  0.00000000e+00, -4.53438249e+02,
       -3.64905032e+03,  0.00000000e+00, -8.76784925e+01,  4.38385447e+01,
        3.50420337e+02, -8.20873935e-02,  1.30328795e+00, -5.06551033e+01,
       -1.49615220e+00, -

Попробуем подобрать гиперпараметры для полиномиальных Риджа и Лассо


In [363]:
ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 8.8}

In [364]:
lasso_optimal = RandomizedSearchCV(Lasso(), parameters).fit(X_train, y_train)
lasso_optimal.best_params_

c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.186e+10, tolerance: 5.552e+07
  model = cd_fast.enet_coordinate_descent(
c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.202e+10, tolerance: 5.478e+07
  model = cd_fast.enet_coordinate_descent(
c:\OmSTU\Python\ml\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.202e+10, tolerance: 

{'alpha': 0.30000000000000004}

In [365]:
ridge = Ridge(alpha=8.8).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 407.07284710893913
MSE: 472534.4316212535
RMSE: 687.4113991062801
MAPE: 0.4024110054694756
R^2: 0.9702749110524977


array([ 0.00000000e+00, -1.51084742e+02,  5.98606560e+01,  3.58554153e+02,
       -3.56243551e+02, -3.30421647e+02, -2.29970248e+02, -3.89003828e+01,
       -1.32886439e+02,  1.01041253e+02,  1.87306593e+01, -1.08092086e+02,
       -1.97093582e+02,  1.03617753e+00,  2.66014503e+01,  2.37970524e+02,
        1.47945971e+02, -7.63000301e+00, -1.30679593e+02,  1.87589151e+02,
        1.73174933e+02, -7.67879716e+01, -3.64850639e+01, -1.44073523e+01,
       -9.47741006e+01, -1.88792517e+03,  1.13428634e-01,  5.81912521e+01,
       -7.36051032e+02,  2.51723653e+03,  1.69923542e+02,  1.59395885e+02,
       -9.67091361e+02,  5.78819552e+02,  4.50049747e+02,  3.08955026e+02,
        5.45160092e+02,  2.51933568e+02, -6.14304376e+02, -8.83485843e+02,
       -1.71599901e+03, -4.29758532e+03,  8.43035950e+02,  1.57665774e+02,
       -1.82597758e+03,  1.07389025e+03,  6.09450549e+02,  1.78399625e+03,
        1.50443938e+03,  7.72166839e-01, -2.49583831e+00,  5.11966973e+00,
        4.27368685e+01, -

In [32]:
lasso = Lasso(alpha=0.3).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 736.7784658834198
MSE: 1288417.63424007
RMSE: 1135.0848577265358
MAPE: 0.6277896943042515
R^2: 0.9189512420334834
0.9189512420334834


array([ 1.12355093e+04, -6.54677211e+01, -2.69735477e+01, -9.94282201e+02,
       -3.05711362e+00, -3.20253364e+01, -7.41074254e+02, -1.54990110e+02,
        1.07557230e+02,  3.01861342e+01,  4.91044640e+02,  2.74126528e+02,
        2.12881852e+02, -5.01364463e+02, -9.79592480e+02, -1.86877384e+03,
       -4.33838279e+03,  9.94140545e+02, -6.85861119e+02, -1.65766940e+03,
        2.14804667e+02, -9.73007671e+01,  6.48668282e+02,  5.92803901e+02])

Лучшая модель: Ридж полинома второго порядка - MAPE меньше, R2 больше


Реализуем метрики:


In [19]:
def MAE(y_pr, y):
    return np.sum(np.absolute(np.array(y_pr) - np.array(y))) / y.size
def MSE(y_pr, y):
    return np.sum(pow(np.array(y_pr) - np.array(y), 2)) / y.size
def RMSE(y_pr, y): 
    return sqrt(MSE(y_pr, y))
def MAPE(y_pr, y):
    return np.sum(np.absolute((np.array(y_pr) - np.array(y)/np.array(y)))) / y.size
def R2(y_pr, y):
    return 1 - y.size * MSE(y_pr, y)/np.sum(pow(np.array(y) - np.mean(np.array(y)), 2))

Реализуем Лассо:


In [70]:
class LinLasso:
    def __init__(self):
        self.y = np.array([])
        
    def fit(self, X, y, learning_rate = 0.0001, max_iter = 100, alpha = 1.):
        X = np.array(X)
        y = np.array(y)
        self.w = np.array([0 for i in range(X.shape[1])])
        self.w = np.append(self.w, 1)
        for t in range(max_iter):
            grad = np.array([])
            for i in range(X.shape[1]):
                grad = np.append(grad, (X[:, i]*(y - X.dot(self.w[:-1]) - self.w[-1])).sum() + alpha * self.w[i])
            grad = np.append(grad,(y - X.dot(self.w[:-1]) - self.w[-1]).sum() + alpha * self.w[-1])
            w_next = self.w - 2 * grad * learning_rate / len(X)
        self.w = w_next
        print(self.w)
        
    def predict(self, X):
        X = np.array(X)
        self.y = X.dot(self.w[:-1]) + self.w[-1]
        return self.y

Проверим Лассо:


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.drop(X_train.columns[[10, 14]], axis=1)
X_test = X_test.drop(X_test.columns[[10, 14]], axis=1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((43152, 24), (43152,), (10788, 24), (10788,))

In [68]:
lasso = LinLasso()
lasso.fit(X_train, y_train)
y_pl = lasso.predict(X_test)
print('MAE - ', MAE(y_pl, y_test))
print('MSE - ', MSE(y_pl, y_test))
print('RMSE - ', RMSE(y_pl, y_test))
print('MAPE - ', MAPE(y_pl, y_test))
print('R2 - ', R2(y_pl, y_test))

[-9.76887042e-01 -4.86285193e+01 -4.54797260e+01 -5.30598550e+00
 -5.30616704e+00 -3.27205426e+00 -2.60907582e-02 -7.17033324e-02
 -2.77499300e-01 -2.33194916e-01 -7.98638209e-02 -1.12083180e-01
 -1.32358537e-01 -1.40766773e-01 -1.01051761e-01 -5.44079996e-02
 -1.05399981e-02 -1.91940443e-02 -1.96110558e-01 -1.71640392e-01
 -1.15955057e-01 -1.77781433e-01 -3.47012375e-02 -6.17754218e-02
  2.12301854e-01]
MAE -  9596.075843504115
MSE -  108197300.6710008
RMSE -  10401.793146905047
MAPE -  5691.040063007267
R2 -  -5.806222300649138
